# テキスト解析の場合

高次元データの例として、テキスト解析をする場合を考えましょう。

データとしてはいつもの 20 newsgroups データを使用することにします。TF-IDF 化まで行われたものを利用できます。

In [ ]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(subset='all', shuffle=True)

テキストをベクトル化するのは前回説明した gensim を使うこともできますが、今回は scikit-learn に用意されているTF-IDFベクトル化を使います。

文書数は18856で、単語は10000語とします。

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000, min_df=2, stop_words='english')
TfIdf = vectorizer.fit_transform(dataset.data)
print(TfIdf.shape)

# KMeans

KMeans法(MiniBatchKMeans)をとりあえず試してみます。

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from time import time
km = MiniBatchKMeans(n_clusters=100, init='k-means++', n_init=1, init_size=1000, batch_size=1000)
# KMeans に切り替えて実行時間を比較するのも試してみてください
#km = KMeans(n_clusters=20, init='k-means++', max_iter=100, n_init=1)
t0 = time()
km.fit(TfIdf)
print("%0.3fs" % (time() - t0))

クラスタリングの性能を評価する指標は幾つかあります。

注意：アイテムが０のクラスタがあると、RuntimeWarning: Mean of empty slice.と出ますが気にせず実行してください。

In [ ]:
from sklearn import metrics
labels = dataset.target # 正解=newsgroup名
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(TfIdf, km.labels_, sample_size=1000))

トピックモデリングを行い次元を削減してみます。今回は `gensim` を使わず scikit-learn の SVD(特異値分解)を使います。クラスタ数は100に固定して、トピック数を変えてSilhouette scoreを比較してみます。(時間がある人はグリッドサーチで、クラスタ数とトピック数をサーチしてみてください。)

（アイテムが０のクラスタがあると、`RuntimeWarning: Mean of empty slice.`と出ますが気にせず実行してください。）

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.grid_search import GridSearchCV
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
n_topics = [2, 5, 10, 20, 50, 100, 200, 500, 1000]
silhouettes = []
for n_topic in n_topics:
    svd = TruncatedSVD(n_topic)
    lsi = make_pipeline(svd, Normalizer(copy=False))
    X = lsi.fit_transform(TfIdf)
    km1 = MiniBatchKMeans(n_clusters=100, init='k-means++', n_init=1, init_size=1000, batch_size=1000)
    km1.fit(X)
    silhouettes.append(metrics.silhouette_score(X, km1.labels_, sample_size=1000))
plt.plot(np.array(n_topics), np.array(silhouettes), 'o-')
plt.xscale('log')

トピック数は100程度あれば十分なようなので、クラスタ数を変えて調べてみます。（アイテムが０のクラスタがあると、`RuntimeWarning: Mean of empty slice.`と出ますが気にせず実行してください。）

In [ ]:
n_clusters = [2, 5, 7, 10, 15, 20, 30, 50, 70, 100, 150, 200, 300, 500, 700, 1000]

svd = TruncatedSVD(100)
lsi = make_pipeline(svd, Normalizer(copy=False))
X = lsi.fit_transform(TfIdf)

silhouettes = []
for n_cluster in n_clusters:
    km1 = MiniBatchKMeans(n_clusters=n_cluster, init='k-means++', n_init=1, init_size=1000, batch_size=1000)
    km1.fit(X)
    silhouettes.append(metrics.silhouette_score(TfIdf, km1.labels_, sample_size=1000))
plt.plot(np.array(n_clusters), np.array(silhouettes), 'o-')
plt.xscale('log')

クラスタ数は大体 100 程度の値で良さそうでしょうか？

# Biclustering

Biclustering は行列を指定した数のブロック対角化された行列へと分解する手法です。

先ほどの解析では文書中の単語をトピックモデリングし、そのトピックへの分解を用いてクラスタリングを行う、という二段階に分けて解析を行いました。Biclustering は（文書）x（単語）という行列の要素が Tf-Idf 値である場合、その行列を、縦も横もトピック数の数だけ分割されたブロック行列へと行列を変形します。

ライブラリの使い方はほぼ同じで `fit` メソッドを呼ぶだけです。

In [ ]:
from sklearn.cluster.bicluster import SpectralCoclustering
cocluster = SpectralCoclustering(n_clusters=20)
cocluster.fit(TfIdf)

データの読み出し方は少し違います。`row_labels_`属性は各データの所属するクラスタ番号を返します。`column_labels_` 属性は素性側の所属するクラスタ番号が入っています。

人間が読みやすいように、`dataset.target_names`を使ってnewsgroup名に、`vectorizer.get_feature_names()`を使って単語に変換します。

結果を見ると、文章と単語とを同時にグループ化しようとしていることが判ります（まぁあまりうまくいってないようにも見えますが。。。）

今回はちゃんとストップワードを指定していないので、意味のなさそうな単語(例えばb8g, 5g9p, 7klj, 9f9, 1fpl, bs0t, 2tct, 7kn, bxn, gizwとか)が辞書に現れているようです。実際の案件ではストップワードなどをちゃんと設定する必要があります。

In [ ]:
print(cocluster.row_labels_)
print(cocluster.row_labels_.shape)
print(len(dataset.target))
print(len(dataset.target_names))
for c in range(20): # c=クラスタ番号
    totalr, dicr = 0, dict()
    totalc, dicc = 0, dict()
    for (i,t) in zip(cocluster.row_labels_, dataset.target):
        name = dataset.target_names[t]
        if (c == i):
            if (name in dicr):
                dicr[name] += 1
            else:
                dicr[name] = 1
            totalr += 1
        else:
            pass
    for (i,name) in zip(cocluster.column_labels_, vectorizer.get_feature_names()):
        if (c == i):
            if (name in dicc):
                dicc[name] += 1
            else:
                dicc[name] = 1
            totalc += 1
        else:
            pass
    print('cluster: {0}: {1},{2}'.format(c, totalr, totalc))
    zsr = [(k,v) for (k,v) in dicr.items()]
    zsr.sort(key=lambda t: -t[1])
    for k,v in zsr[:3]:
        if (totalc != 0):
            print('   {0} : {1:.2f}%'.format(k, v/totalc*100))
        else:
            pass
    zsc = [(k,v) for (k,v) in dicc.items()]
    zsc.sort(key=lambda t: -t[1])
    print('   words={0}'.format(','.join([k for k,v in zsc[:10]])))



今回やっていることは、トピックモデリングと似たことを繰り返したように見えます。顧客と購入アイテムの行列などを用いればレコメンデーションや顧客クラスタリングなどにも使用可能でしょう。

あるいは Tf-Idf 行列の代わりにノード間の近接度を表す行列を使えば、ノードのクラスタリングが行われます。

色々、使い方を考えてみてください。